In [ ]:
import numpy as np
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My\ Drive/DeepLearning/ZaloChallenge/za_traffic_2020/

/content/gdrive/My Drive/DeepLearning/ZaloChallenge/za_traffic_2020


In [ ]:
!curl -LJO https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-custom.cfg
!curl -LJO https://raw.githubusercontent.com/AlexeyAB/darknet/master/Makefile

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12260  100 12260    0     0  49635      0 --:--:-- --:--:-- --:--:-- 49635
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5806  100  5806    0     0  15565      0 --:--:-- --:--:-- --:--:-- 15524


In [ ]:
!git clone https://github.com/AlexeyAB/darknet
!rm -rf data
!mkdir data

Cloning into 'darknet'...
remote: Enumerating objects: 14390, done.
remote: Total 14390 (delta 0), reused 0 (delta 0), pack-reused 14390
Receiving objects: 100% (14390/14390), 13.10 MiB | 4.72 MiB/s, done.
Resolving deltas: 100% (9781/9781), done.
Checking out files: 100% (2021/2021), done.
/content/gdrive/My Drive/DeepLearning/ZaloChallenge/za_traffic_2020/darknet


In [ ]:
%cd ./darknet/

/content/gdrive/My Drive/DeepLearning/ZaloChallenge/za_traffic_2020/darknet


In [ ]:
!echo -e "Cam nguoc chieu \nCam dung va do \nCam re \nGioi han toc do \nCam con lai \nNguy hiem \nHieu lenh" >> label.names

In [ ]:
!mkdir backup
!echo classes=7 > yolo.data
!echo train=train.txt >> yolo.data
!echo valid=val.txt >> yolo.data
!echo names=label.names >> yolo.data
!echo backup=backup >> yolo.data

In [ ]:
import json 
import glob 

# Read json label of train dataset
with open('/content/gdrive/MyDrive/DeepLearning/ZaloChallenge/za_traffic_2020/darknet/data/traffic_train/train_traffic_sign_dataset.json', 'r') as f:
  data = json.load(f)

# All train images path
imgs_path = glob.glob('/content/gdrive/MyDrive/DeepLearning/ZaloChallenge/za_traffic_2020/darknet/data/traffic_train/images/*.png')

bboxs_list = data['annotations']
categories = data['categories']
imgs_info = data['images']

In [ ]:
# Function to remove same box detection in training data
def samebox(last_el, arr):
  tmp1 = last_el[1], last_el[1] + last_el[3], last_el[2], last_el[2] + last_el[4]
  lb1 = last_el[0]
  for i in arr:
    lb2 = i[0]
    tmp2 = i[1], i[1] + i[3], i[2], i[2] + i[4]
    if abs(tmp1[0] - tmp2[0]) * IMG_WIDTH < 5 and abs(tmp1[1] - tmp2[1]) * IMG_HEIGHT < 5 and abs(tmp1[2] - tmp2[2]) * IMG_WIDTH < 5 and abs(tmp1[3] - tmp2[3]) * IMG_HEIGHT < 5:
      print(tmp1, tmp2, lb1, lb2)
      return True
  return False 

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 1622, 626
idx2bbox = {}
for x in bboxs_list:
  if x['image_id'] not in idx2bbox:
    idx2bbox[x['image_id']] = []
  bbox = x['bbox'] 

  # Remove box with width and height smaller than 6 pixel 
  if len(idx2bbox[x['image_id']]) == 0 or ((bbox[2] > 5) and (bbox[3] > 5)):
    # Add 1 pixel left and 2 pixel bottom 
    # x_center = left + width/2, y_center = top + height/2 
    x_center, y_center = round((bbox[0] - 1 + (bbox[2] + 1)/2)/IMG_WIDTH, 6), round((bbox[1] + (bbox[3] + 2)/2)/IMG_HEIGHT, 6)
    width, height = round((bbox[2] + 1)/IMG_WIDTH, 6), round((bbox[3] + 2)/IMG_HEIGHT, 6)
    idx2bbox[x['image_id']].append([x['category_id'] - 1, x_center, y_center,
                                    width, height                               
                                   ])
  
  if len(idx2bbox[x['image_id']]) > 1 and samebox(idx2bbox[x['image_id']][-1], idx2bbox[x['image_id']][:-1]):
      idx2bbox[x['image_id']].pop() 

# idx2bbox = {str(x['id']): x['bbox'] for x in bbox}
# idx2bbox

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 1622, 626
idx2bbox = {}
for x in bboxs_list:
  if x['image_id'] not in idx2bbox:
    idx2bbox[x['image_id']] = []
  bbox = x['bbox'] 

  # Remove box with width and height smaller than 6 pixel 
    # Add 1 pixel left and 2 pixel bottom 
    # x_center = left + width/2, y_center = top + height/2 
  idx2bbox[x['image_id']].append([x['category_id'] - 1, bbox[0], bbox[1], bbox[2], bbox[3]])
  # print(bbox)

In [ ]:
import os 
import time

def save_label_to_text(img_folder, X_dict):
  count, len_X = 0, len(X_dict)
  for i, bboxs in X_dict.items():
    count += 1
    if count % 450 == 0:
      print(f'{count / len_X * 100} %')
    string = ''
    for i_bbox in bboxs:
      string_bbox = map(str, i_bbox)  
      string += ' '.join(string_bbox) 
      string += '\n' 
    with open(os.path.join(img_folder, f'{i}.txt'), 'w') as f:
      f.write(string) 

start = time.time()
save_label_to_text( '/content/gdrive/MyDrive/DeepLearning/ZaloChallenge/za_traffic_2020/darknet/data/traffic_train/images', idx2bbox)
print(f'Save object file done ! Total time : {time.time()-start}.')

In [ ]:
idxs = np.arange(len(imgs_path))
np.random.shuffle(idxs)

In [ ]:
len_train = int(0.8*len(imgs_path))
train_string = ''
val_string = ''
for i in imgs_path[:len_train]:
  train_string += f'{i}\n'
for i in imgs_path[len_train:]:
  val_string += f'{i}\n'

In [ ]:
def save_file(file_name, string):
  with open(file_name, 'w') as f:
    f.write(string)

save_file('train.txt', train_string)
save_file('val.txt', val_string)

In [ ]:
      #    CREATE test.txt WITH ALL IMAGES PATH    #
import glob 

test_imgs_path = glob.glob('/content/gdrive/MyDrive/DeepLearning/ZaloChallenge/za_traffic_2020/darknet/data/traffic_public_test/images/*.png')
test_string = ''

for i, name  in enumerate(test_imgs_path):
  test_string += f'{name}\n'
    
save_file('test.txt', test_string)

586


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
count_list = [len(x) for x in idx2bbox.values()]
count = 0
for x in count_list:
  count += x
count_list.index(3)
# print(count)

In [ ]:
import cv2
import math

IMG_WIDTH, IMG_HEIGHT = 1622, 626


def get_bbox(filepath):
  img = cv2.imread(filepath)
  img_name = filepath.split('/')[-1]
  idx_img = int(img_name.split('.')[0])
  bboxs = idx2bbox[idx_img]

  for bbox in bboxs:
    top_left = (int(math.ceil((bbox[1]-bbox[3]/2)*IMG_WIDTH)),
                 int(math.ceil((bbox[2]-bbox[4]/2)*IMG_HEIGHT)))
    bottom_right = (int(math.ceil((bbox[1]+bbox[3]/2)*IMG_WIDTH)),
                    int(math.ceil((bbox[2]+bbox[4]/2)*IMG_HEIGHT)))
    color, thickness = (0, 255, 0), 1
    img = cv2.rectangle(img, top_left, bottom_right, color, thickness) 

  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
  plt.figure(figsize=(60, 60))
  plt.imshow(img)

  
get_bbox('/content/gdrive/MyDrive/DeepLearning/ZaloChallenge/za_traffic_2020/darknet/data/traffic_train/images/11098.png')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import cv2
import math


def get_topleft_bbox(filepath):
  img = cv2.imread(filepath)
  img_name = filepath.split('/')[-1]
  idx_img = int(img_name.split('.')[0])
  bboxs = idx2bbox[idx_img]

  for bbox in bboxs:
    print(bbox)
    top_left = (bbox[1], bbox[2])
    bottom_right = (bbox[1] + bbox[3], bbox[2] + bbox[4])
    color = (0, 255, 0) 
    thickness = 1
    img = cv2.rectangle(img, top_left, bottom_right, color, thickness) 

  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
  plt.figure(figsize=(60, 60))
  plt.imshow(img)

  
get_topleft_bbox(imgs_path[4])

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
plt.figure(figsize=(50, 50))
get_bbox(imgs_path[0])

Output hidden; open in https://colab.research.google.com to view.